# NLLB Full Fine-Tuning and Evaluation
## Objective
The objective of this notebook is to execute a core experimental workflow of the thesis: to **fully fine-tune** the state-of-the-art NLLB model on a custom bidirectional corpus and conduct a comprehensive evaluation to measure its performance against a zero-shot baseline. This script serves as a self-contained experiment, generating the quantitative metrics and visualizations necessary to validate the effectiveness of the full fine-tuning adaptation strategy.

## Methodology
The notebook follows a rigorous, sequential experimental design:

1. **Data Preparation:** Loads the structured, bidirectional `.jsonl` corpus, splits it into training (80%), validation (10%), and test (10%) sets, and tokenizes the text for the NLLB model.
2. **Baseline Evaluation:** Establishes a performance benchmark by evaluating the original, pre-trained NLLB model on the held-out test set for both `Odia → German` and `German → Odia` directions.
3. **Full Fine-Tuning:** Trains the model on the bidirectional corpus, updating all of its 617 million parameters to specialize it for the new task.
4. **Final Evaluation:** Evaluates the newly fine-tuned model on the same held-out test set.
5. **Results Synthesis:** Consolidates all performance metrics (BLEU, chrF, TER) into a final comparison table and generates visualizations to analyze the results.

## Workflow
1. Mounts Google Drive for data access.

2. Installs all required libraries (`transformers`, `datasets`, etc.).

3. Loads and splits the `bidirectional_corpus.jsonl` file.

4. Tokenizes the datasets.

5. Performs the baseline evaluation.

6. Executes the Full Fine-Tuning training run.

7. Performs the final evaluation on the fine-tuned model.

8. Generates and displays the final summary table and all visualizations comparing the Baseline and the Fully Fine-Tuned model.

## Input & Output
* **Input:** A single `.jsonl` file (`bidirectional_corpus.jsonl`).
* **Output:**
  * Saved model artifacts for the Fully Fine-Tuned model in a specified Google Drive directory.
  * A printed summary table comparing the performance of the Baseline and the Fully Fine-Tuned model.
  * A series of plots visualizing the final performance comparison and training dynamics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q transformers datasets sacrebleu torch accelerate pandas bitsandbytes

In [ ]:
print("--- All Installed Packages (pip list) ---")
!pip list

In [ ]:
# import libraries
import json
import torch
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sacrebleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline
from tqdm.auto import tqdm

# Configuration and Data Preparation

This code block sets up all parameters, loads your dataset, and splits it into the crucial train, validation, and test sets.

In [ ]:
# --- CONFIGURATION ---

# --- File and Model Configuration ---
CORPUS_FILE = "/content/drive/MyDrive/Thesis/data/transformed/bidirectional_corpus_final.jsonl"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
OUTPUT_DIR = "/content/drive/MyDrive/Thesis/eval/nllb-finetuned-odia-german-model-results"
FINAL_MODEL_PATH = "/content/drive/MyDrive/Thesis/model/nllb-odia-german-translator_model_final"

# --- Field Names ---
INPUT_FIELD = "input_text" # As defined in the JSONL
TARGET_FIELD = "target_text" # As defined in the JSONL

# --- Language Configuration ---
ODIA_LANG_CODE = "ory_Orya"
GERMAN_LANG_CODE = "deu_Latn"

# --- Split Configuration ---
TEST_SET_SIZE = 0.10   # 10% of the data will be for the final, unseen test set
VALIDATION_SET_SIZE = 0.10 # 10% of the remaining data for validation

# --- Task Prefixes ---
PREFIX_ORI_TO_DEU = "translate Odia to German: "
PREFIX_DEU_TO_ORI = "translate German to Odia: "

In [ ]:
# --- LOAD AND SPLIT THE DATASET ---

print("Loading the rich bidirectional dataset...")

# --- Manually load the JSONL file into a list of dictionaries ---
data_list = []
with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

# --- Create a pandas DataFrame ---
df = pd.DataFrame(data_list)

# --- Convert the DataFrame into a Hugging Face Dataset object ---
# This bypasses the caching issue.
full_dataset = Dataset.from_pandas(df)

print("Dataset loaded successfully.")

# --- Shuffle the dataset before splitting ---
full_dataset = full_dataset.shuffle(seed=42)

# --- Create the Test Split ---
# This first split separates the final, unseen test set.
train_valid_split = full_dataset.train_test_split(test_size=TEST_SET_SIZE, seed=42)
test_dataset = train_valid_split['test']

# --- Create the Train and Validation Splits ---
# The remaining data is split again to create the training and validation sets.
train_val_split = train_valid_split['train'].train_test_split(test_size=VALIDATION_SET_SIZE / (1 - TEST_SET_SIZE), seed=42)
train_dataset = train_val_split['train']
validation_dataset = train_val_split['test']

# Combine into a single DatasetDict for convenience
split_datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

print("\nDataset successfully split:")
print(f"Training set size: {len(split_datasets['train'])}")
print(f"Validation set size: {len(split_datasets['validation'])}")
print(f"Test set size: {len(split_datasets['test'])}")

In [ ]:
# --- Corpus Analysis (EDA) ---

print("\n--- Starting Corpus Analysis (EDA) ---")

# --- Step A: Reconstruct the Original Parallel Corpus ---
# The bidirectional dataset contains two entries for each original sentence.
# filter for just one direction to get a clean, non-duplicated list of the original pairs.
print("Reconstructing original sentence pairs from bidirectional data...")

# Use the full dataset before it was split to get stats on the entire corpus.
odia_source_sentences = [
    ex[INPUT_FIELD].replace(PREFIX_ORI_TO_DEU, "")
    for ex in full_dataset
    if ex[INPUT_FIELD].startswith(PREFIX_ORI_TO_DEU)
]
german_target_sentences = [
    ex[TARGET_FIELD]
    for ex in full_dataset
    if ex[INPUT_FIELD].startswith(PREFIX_ORI_TO_DEU)
]

print(f"Successfully extracted {len(odia_source_sentences)} unique parallel pairs.")


# --- Step B: Calculate Sentence Lengths ---
# Calculate the number of words by splitting on spaces.
odia_lengths = [len(s.split()) for s in odia_source_sentences]
german_lengths = [len(s.split()) for s in german_target_sentences]

In [ ]:
# --- Step C: Calculate and Display Descriptive Statistics ---
stats_data = {
    'Language': ['Odia', 'German'],
    'Mean Length': [np.mean(odia_lengths), np.mean(german_lengths)],
    'Median Length': [np.median(odia_lengths), np.median(german_lengths)],
    'Std. Deviation': [np.std(odia_lengths), np.std(german_lengths)],
    'Max Length': [np.max(odia_lengths), np.max(german_lengths)]
}
df_stats = pd.DataFrame(stats_data).set_index('Language')

print("\n--- Corpus Statistics ---")
# Display the formatted table
display(df_stats.style.format("{:.2f}"))

In [ ]:
# --- Step D: Visualize Sentence Length Distribution ---
print("\n--- Generating Sentence Length Distribution Plot ---")

# Prepare data for plotting with Seaborn
plot_df_odia = pd.DataFrame({'Sentence Length (words)': odia_lengths})
plot_df_odia['Language'] = 'Odia'
plot_df_german = pd.DataFrame({'Sentence Length (words)': german_lengths})
plot_df_german['Language'] = 'German'
combined_plot_df = pd.concat([plot_df_odia, plot_df_german])

# Create the histogram plot
sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
fig.suptitle('Figure 4.1: Distribution of Sentence Lengths in the Corpus', fontsize=18, y=1.03)

# Plot for Odia
sns.histplot(data=plot_df_odia, x='Sentence Length (words)', ax=axes[0], kde=True, bins=30)
axes[0].set_title('Odia Sentence Lengths', fontsize=14)
axes[0].set_xlabel('Length (in words)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)

# Plot for German
sns.histplot(data=plot_df_german, x='Sentence Length (words)', ax=axes[1], kde=True, bins=30, color='orange')
axes[1].set_title('German Sentence Lengths', fontsize=14)
axes[1].set_xlabel('Length (in words)', fontsize=12)
axes[1].set_ylabel('') # No need for a second y-axis label

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("\n✅ Corpus Analysis complete.")

In [ ]:
# --- Vocabulary Analysis ---
def calculate_vocab_size(sentences_list, tokenizer):
  """
  Calculates the number of unique subword tokens in a list of sentences using a tokenizer.

  This function tokenizes each sentence in the provided list using the specified tokenizer,
  collects all subword tokens into a set to ensure uniqueness, and returns the size of the
  resulting vocabulary. A progress bar is displayed using `tqdm` to track tokenization for
  large datasets.

  Args:
    sentences_list (list[str]): A list of sentences to be tokenized.
    tokenizer (object): A tokenizer object with a `tokenize` method that splits a sentence
    into a list of subword tokens (e.g., a Hugging Face tokenizer).

  Returns:
    int: The number of unique subword tokens in the tokenized sentences.

  Note:
    - Requires the `tqdm` library for progress bar visualization.
    - Uses a set for efficient storage and counting of unique tokens.
    - Suitable for preprocessing tasks in NLP, such as building vocabularies for machine translation models.
  """
  # Using a set is a highly efficient way to store and count unique items.
  unique_tokens = set()

  print(f"Tokenizing {len(sentences_list)} sentences...")
  # Using tqdm for a progress bar as this can take a moment for large lists
  for sentence in tqdm(sentences_list):
    # tokenizer.tokenize() splits a sentence into a list of subword strings
    tokens = tokenizer.tokenize(sentence)
    # .update() adds all items from the list to the set
    unique_tokens.update(tokens)

  return len(unique_tokens)

# --- Calculate for Odia ---
print("\nAnalyzing Odia corpus...")
odia_vocab_size = calculate_vocab_size(odia_source_sentences, tokenizer)

# --- Calculate for German ---
print("\nAnalyzing German corpus...")
german_vocab_size = calculate_vocab_size(german_target_sentences, tokenizer)

# --- Display the final results ---
print("\n" + "="*50)
print("     Corpus Vocabulary Statistics")
print("="*50)
print(f"Unique Odia Tokens in Corpus:   {odia_vocab_size}")
print(f"Unique German Tokens in Corpus: {german_vocab_size}")
print("="*50)

In [ ]:
# --- 3. PREPARE TOKENIZER AND PREPROCESSING FUNCTION ---

print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang=ODIA_LANG_CODE, tgt_lang=GERMAN_LANG_CODE)

def preprocess_function(examples):
  """
  Tokenizes input and target text fields for model training or evaluation.

  This function processes a batch of examples from a dataset, tokenizing the `input_text` and
  `target_text` fields using a provided tokenizer. It applies truncation to a maximum length of
  1200 tokens and generates input IDs for both inputs and labels, with the latter prepared in the
  tokenizer's target context. The resulting dictionary includes tokenized inputs and labels suitable
  for training or evaluating a translation model.

  Args:
    examples (dict): A dictionary containing lists of input and target sentences, with keys
                     defined by `INPUT_FIELD` and `TARGET_FIELD` (e.g., 'input_text' and 'target_text').

  Returns:
    dict: A dictionary containing tokenized inputs (`input_ids`, etc.) and labels
          (`labels` as input IDs from target text tokenization).

  Note:
    - Assumes global variables `INPUT_FIELD`, `TARGET_FIELD`, and `tokenizer` are defined.
    - Requires a tokenizer compatible with Hugging Face's `transformers` library.
    - The maximum length of 1200 tokens is hardcoded for truncation.

  Example:
    >>> from transformers import AutoTokenizer
    >>> tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
    >>> INPUT_FIELD = "input_text"
    >>> TARGET_FIELD = "target_text"
    >>> examples = {
    ...     "input_text": ["ନମସ୍କାର"],
    ...     "target_text": ["Hallo"]
    ... }
    >>> result = preprocess_function(examples)
    >>> print(result.keys())
    dict_keys(['input_ids', 'attention_mask', 'labels'])
  """
  # Access the lists of sentences directly from the batch columns
  inputs = examples[INPUT_FIELD]
  targets = examples[TARGET_FIELD]

  model_inputs = tokenizer(inputs, max_length=1200, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=1200, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

print("Tokenizing all dataset splits...")

tokenized_datasets = split_datasets.map(preprocess_function, batched=True, batch_size=16, remove_columns=split_datasets['train'].column_names)
print("Tokenization complete.")

# Baseline Evaluation

This code block evaluates the original NLLB model on the `test` set before any fine-tuning happens.

In [ ]:
# --- EVALUATION LOGIC ---
def evaluate_model_on_task(model_to_eval, tokenizer, test_data, task_prefix, src_lang, tgt_lang, batch_size=8):
  """
  Evaluates a translation model on a specific translation direction using a test dataset.

  This function filters test data for sentences starting with a specified task prefix, generates
  translations using a translation pipeline, and computes BLEU, CHRF++, and TER scores against
  reference translations. It processes the data in batches to manage memory and displays progress
  using a `tqdm` progress bar. If no data matches the prefix, it returns zero scores.

  Args:
    model_to_eval (object): The translation model to evaluate (e.g., a Hugging Face model).
    tokenizer (object): The tokenizer associated with the model.
    test_data (list[dict]): A list of dictionaries containing input and target sentences,
                            with keys defined by `INPUT_FIELD` and `TARGET_FIELD`.
    task_prefix (str): A prefix to filter test data for a specific translation direction (e.g., '[ORI_TO_DEU]').
    src_lang (str): Source language code (e.g., 'ory_Orya').
    tgt_lang (str): Target language code (e.g., 'deu_Latn').
    batch_size (int, optional): Number of sentences to process per batch. Defaults to 8.

  Returns:
    dict: A dictionary containing evaluation metrics:
          - 'bleu': BLEU score.
          - 'chrf': CHRF++ score (word order 2).
          - 'ter': TER score.
  """
  # Filter the test set for the specific task using the full prefix
  task_inputs = [ex[INPUT_FIELD] for ex in test_data if ex[INPUT_FIELD].startswith(task_prefix)]
  task_references = [[ex[TARGET_FIELD]] for ex in test_data if ex[INPUT_FIELD].startswith(task_prefix)]

  if not task_inputs:
    print(f"Warning: No test data found for prefix '{task_prefix}'")
    return {"bleu": 0, "chrf": 0, "ter": 0}

  device = 0 if torch.cuda.is_available() else -1
  translator = pipeline("translation", model=model_to_eval, tokenizer=tokenizer, src_lang=src_lang, tgt_lang=tgt_lang)

  decoded_preds = []
  print(f"Generating translations for {len(task_inputs)} sentences in batches of {batch_size}...")
  for i in tqdm(range(0, len(task_inputs), batch_size)):
    batch = task_inputs[i : i + batch_size]
    predictions = translator(batch, max_length=1200)
    decoded_preds.extend([pred['translation_text'] for pred in predictions])

  bleu = sacrebleu.BLEU().corpus_score(decoded_preds, task_references).score
  chrf = sacrebleu.CHRF(word_order=2).corpus_score(decoded_preds, task_references).score
  ter = sacrebleu.TER().corpus_score(decoded_preds, task_references).score

  return {"bleu": bleu, "chrf": chrf, "ter": ter}

In [ ]:
# --- BASELINE EVALUATION ---
from transformers import BitsAndBytesConfig

print("\n--- Evaluating Baseline Model (Before Fine-Tuning) ---")
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME, quantization_config=bnb_config, device_map="auto"
)

print("Evaluating Odia -> German baseline...")
baseline_ori_deu_metrics = evaluate_model_on_task(base_model, tokenizer, split_datasets['test'], PREFIX_ORI_TO_DEU, ODIA_LANG_CODE, GERMAN_LANG_CODE)

print("Evaluating German -> Odia baseline...")
baseline_deu_ori_metrics = evaluate_model_on_task(base_model, tokenizer, split_datasets['test'], PREFIX_DEU_TO_ORI, GERMAN_LANG_CODE, ODIA_LANG_CODE)

In [ ]:
# Clear memory before training
del base_model
torch.cuda.empty_cache()

# Full Fine-Tuning

This code block sets up and runs the actual fine-tuning process.

In [ ]:
# --- FULL FINE-TUNING THE MODEL ---
print("\n--- Starting Fine-Tuning Process ---")
# Reload the model without quantization for training
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

def compute_metrics_for_trainer(eval_preds):
  """
  Computes BLEU score for model evaluation during training.

  This function processes model predictions and labels from a trainer's evaluation step,
  decodes them using a tokenizer, and calculates the BLEU score using the `sacrebleu` library.
  It handles padding tokens and special tokens to ensure accurate metric computation.

  Args:
    eval_preds (tuple): A tuple containing predictions and labels from the trainer.
                        Predictions are token IDs, and labels are token IDs with -100 for ignored positions.

  Returns:
    dict: A dictionary containing the BLEU score with key 'bleu'.

  Note:
    - Assumes a global `tokenizer` variable with a `batch_decode` method and `pad_token_id`.
    - Requires `sacrebleu` and `numpy` libraries.
    - Designed for use with Hugging Face's `transformers` Trainer API.

  Example:
    >>> from transformers import AutoTokenizer
    >>> tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
    >>> eval_preds = (
    ...     tokenizer(["Hallo"], return_tensors="pt")["input_ids"].numpy(),
    ...     tokenizer(["Hallo"], return_tensors="pt")["input_ids"].numpy()
    ... )
    >>> result = compute_metrics_for_trainer(eval_preds)
    >>> print(result)
    {'bleu': 100.0}
  """
  preds, labels = eval_preds
  if isinstance(preds, tuple): preds = preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  decoded_labels = [[label] for label in decoded_labels]
  result = sacrebleu.BLEU().corpus_score(decoded_preds, decoded_labels)
  return {"bleu": result.score}

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    optim="adafactor",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
    compute_metrics=compute_metrics_for_trainer,
)

trainer.train()

# Final Evaluation and Comparison

This code block evaluates your newly fine-tuned model on the same test set and presents a clear comparison with the baseline.

In [ ]:
# --- FINAL EVALUATION ---
print("\n--- Evaluating Fine-Tuned Model ---")
# Load the fine-tuned model in 8-bit mode for memory-efficient evaluation
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(trainer.state.best_model_checkpoint, load_in_8bit=True, device_map="auto")

print("Evaluating Odia -> German fine-tuned...")
finetuned_ori_deu_metrics = evaluate_model_on_task(finetuned_model, tokenizer, split_datasets['test'], PREFIX_ORI_TO_DEU, ODIA_LANG_CODE, GERMAN_LANG_CODE)

print("Evaluating German -> Odia fine-tuned...")
finetuned_deu_ori_metrics = evaluate_model_on_task(finetuned_model, tokenizer, split_datasets['test'], PREFIX_DEU_TO_ORI, GERMAN_LANG_CODE, ODIA_LANG_CODE)

In [ ]:
# --- DISPLAY FINAL RESULTS TABLE ---
results_data = {
    'Metric': ['BLEU', 'chrF', 'TER (lower is better)'],
    'Odia → German (Baseline)': [baseline_ori_deu_metrics['bleu'], baseline_ori_deu_metrics['chrf'], baseline_ori_deu_metrics['ter']],
    'Odia → German (Fine-Tuned)': [finetuned_ori_deu_metrics['bleu'], finetuned_ori_deu_metrics['chrf'], finetuned_ori_deu_metrics['ter']],
    'German → Odia (Baseline)': [baseline_deu_ori_metrics['bleu'], baseline_deu_ori_metrics['chrf'], baseline_deu_ori_metrics['ter']],
    'German → Odia (Fine-Tuned)': [finetuned_deu_ori_metrics['bleu'], finetuned_deu_ori_metrics['chrf'], finetuned_deu_ori_metrics['ter']],
}

df_results = pd.DataFrame(results_data).set_index('Metric')
print("\n\n" + "="*80)
print("                           FINAL PERFORMANCE RESULTS")
print("="*80)
display(df_results.style.format("{:.4f}"))
print("="*80)

In [ ]:
# --- VISUALIZATION: Training and Validation Progress ---

print("--- Generating Training and Validation Curves ---")

# The trainer object holds the history of the training run
# Extract the logs to a pandas DataFrame for easy plotting
try:
  log_history = trainer.state.log_history
  df_logs = pd.DataFrame(log_history)

  # Separate training and evaluation logs
  df_train = df_logs[df_logs['loss'].notna()].copy()
  df_eval = df_logs[df_logs['eval_loss'].notna()].copy()

  # Create the plot with two y-axes
  sns.set_theme(style="whitegrid")
  fig, ax1 = plt.subplots(figsize=(12, 7))

  # Plot Training and Validation Loss on the primary y-axis (ax1)
  ax1.set_xlabel('Training Step', fontsize=12)
  ax1.set_ylabel('Loss', color='C0', fontsize=14)
  ax1.plot(df_train['step'], df_train['loss'], color='C0', marker='.', linestyle=':', label='Training Loss')
  ax1.plot(df_eval['step'], df_eval['eval_loss'], color='C0', marker='o', linestyle='-', label='Validation Loss')
  ax1.tick_params(axis='y', labelcolor='C0')
  ax1.legend(loc='upper left')

  # Create a second y-axis for the BLEU score
  ax2 = ax1.twinx()
  ax2.set_ylabel('Validation BLEU Score', color='C1', fontsize=14)
  ax2.plot(df_eval['step'], df_eval['eval_bleu'], color='C1', marker='s', linestyle='--', label='Validation BLEU')
  ax2.tick_params(axis='y', labelcolor='C1')
  ax2.legend(loc='upper right')

  plt.title('Model Training and Validation Progress', fontsize=18, pad=20)
  fig.tight_layout()
  plt.show()

except NameError:
  print("\n⛔️ ERROR: The 'trainer' object was not found.")
  print("Please ensure you have run the main Fine-Tuning cell successfully before running this visualization cell.")
except Exception as e:
  print(f"\nAn error occurred: {e}")

This chart visualizes the model's learning process over the course of the fine-tuning experiment. It provides two key pieces of evidence regarding the success and stability of the training process, directly supporting your methodology.

**1. The Proof of Learning: Consistently Decreasing Loss**

The most important story on this plot is told by the blue lines, which represent the model's loss.

* **Validation Loss (Solid Blue Line):** This is the most critical curve. Its consistent downward trend across the training steps (from an initial value of ~1.32 down to ~1.28) is the definitive indicator that the model was **successfully learning and generalizing**. A decreasing loss on the validation set proves that the model was not simply memorizing the training data but was learning underlying linguistic patterns that allowed it to become more accurate on unseen data. This is strong evidence against overfitting.
* **Training Loss (Dotted Blue Line):** The corresponding steep decrease in training loss confirms that the optimization algorithm (`adafactor`) was working correctly, effectively minimizing the error on the data the model was being trained on.

Hence, the validation loss consistently decreased throughout the training process, indicating that the model was successfully learning generalizable patterns from the training corpus and was not overfitting. The stable convergence confirms the viability of the chosen hyperparameter configuration and fine-tuning strategy.

**2. The Nuanced Finding: A Stable (Flat) Validation BLEU Score**

* The BLEU score on the validation set starts high (around 40.5) and remains remarkably stable throughout the training process.
* At first glance, a flat BLEU score might seem to contradict the decreasing loss. However, this is a common and insightful finding in NLP. It does not mean the model wasn't learning. It means:

1. **High Initial Performance:** The model likely achieved a very good translation quality on the validation set early in the training (by the first evaluation step).

2. **Subtle Improvements:** The decreasing validation loss shows the model was becoming progressively more confident in its predictions and was refining the probabilities it assigned to the correct words. However, these improvements in probabilistic accuracy were not significant enough to change the final generated words in a way that would alter the n-gram overlap measured by BLEU. The model was essentially learning to perfect its already good predictions, a subtle improvement that a coarse metric like BLEU may not be sensitive enough to capture epoch-over-epoch.

3. **Validation Set Composition:** It's also possible that the validation set, while good, did not contain enough linguistic diversity to reflect these subtle improvements in a changing BLEU score.

Hence, an interesting finding from the training process is the stability of the validation BLEU score, which remained high at approximately 40.5 across all evaluation steps. This apparent plateau, when contrasted with the steadily decreasing validation loss, suggests that the model achieved a high level of translation quality early in the fine-tuning process. The subsequent training epochs served to refine and solidify this knowledge, increasing the model's predictive confidence (as shown by the lower loss) without making surface-level changes that would significantly alter the n-gram-based BLEU metric. This highlights a known limitation of discrete evaluation metrics and underscores the value of using validation loss as a primary indicator of successful model convergence. It proves your model learned correctly and didn't overfit.

In [ ]:
# --- VISUALIZATION: Score Distribution (KDE Plot) ---

print("\n--- Generating Score Distribution Visualizations ---")

# Helper function to generate predictions for this specific analysis
def generate_predictions_for_viz(model, tokenizer, inputs, src_lang, tgt_lang, batch_size=8):
  """
  Generates translations for a list of input sentences for visualization or analysis.

  This function uses a translation pipeline to generate translations for a list of input sentences,
  processing them in batches to manage memory. It is designed for visualization or analysis tasks,
  such as inspecting model outputs. Progress is displayed using a `tqdm` progress bar.

  Args:
    model (object): The translation model (e.g., a Hugging Face model).
    tokenizer (object): The tokenizer associated with the model.
    inputs (list[str]): A list of input sentences to translate.
    src_lang (str): Source language code (e.g., 'ory_Orya').
    tgt_lang (str): Target language code (e.g., 'deu_Latn').
    batch_size (int, optional): Number of sentences to process per batch. Defaults to 8.

  Returns:
    list[str]: A list of translated sentences.
  """
  translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang=tgt_lang)
  predictions = []
  for i in tqdm(range(0, len(inputs), batch_size), desc=f"Translating for Viz"):
    batch = inputs[i : i + batch_size]
    preds = translator(batch, max_length=1200)
    predictions.extend([p['translation_text'] for p in preds])
  return predictions

# --- Generate predictions for both models and both directions ---
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Baseline Predictions
print("\nLoading BASELINE model...")
base_model_viz = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, device_map="auto")
baseline_preds_od = generate_predictions_for_viz(base_model_viz, tokenizer, [ex[INPUT_FIELD] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_ORI_TO_DEU)], ODIA_LANG_CODE, GERMAN_LANG_CODE)
baseline_preds_do = generate_predictions_for_viz(base_model_viz, tokenizer, [ex[INPUT_FIELD] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_DEU_TO_ORI)], GERMAN_LANG_CODE, ODIA_LANG_CODE)
del base_model_viz
torch.cuda.empty_cache()

# Fine-Tuned Predictions
print("\nLoading FINE-TUNED model...")
finetuned_model_viz = AutoModelForSeq2SeqLM.from_pretrained(FINAL_MODEL_PATH, quantization_config=bnb_config, device_map="auto")
finetuned_preds_od = generate_predictions_for_viz(finetuned_model_viz, tokenizer, [ex[INPUT_FIELD] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_ORI_TO_DEU)], ODIA_LANG_CODE, GERMAN_LANG_CODE)
finetuned_preds_do = generate_predictions_for_viz(finetuned_model_viz, tokenizer, [ex[INPUT_FIELD] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_DEU_TO_ORI)], GERMAN_LANG_CODE, ODIA_LANG_CODE)
del finetuned_model_viz
torch.cuda.empty_cache()

# --- Calculate sentence-level BLEU scores ---
sbleu = sacrebleu.BLEU(effective_order=True)

# For Odia -> German
refs_od = [[ex[TARGET_FIELD]] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_ORI_TO_DEU)]
baseline_scores_od = [sbleu.sentence_score(pred, ref).score for pred, ref in zip(baseline_preds_od, refs_od)]
finetuned_scores_od = [sbleu.sentence_score(pred, ref).score for pred, ref in zip(finetuned_preds_od, refs_od)]
df_dist_od = pd.DataFrame({'Baseline': baseline_scores_od, 'Fine-Tuned': finetuned_scores_od})

# For German -> Odia
refs_do = [[ex[TARGET_FIELD]] for ex in split_datasets['test'] if ex[INPUT_FIELD].startswith(PREFIX_DEU_TO_ORI)]
baseline_scores_do = [sbleu.sentence_score(pred, ref).score for pred, ref in zip(baseline_preds_do, refs_do)]
finetuned_scores_do = [sbleu.sentence_score(pred, ref).score for pred, ref in zip(finetuned_preds_do, refs_do)]
df_dist_do = pd.DataFrame({'Baseline': baseline_scores_do, 'Fine-Tuned': finetuned_scores_do})

# --- Create the plots ---
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
sns.set_theme(style="whitegrid")
fig.suptitle('Distribution of Sentence-Level BLEU Scores by Translation Direction', fontsize=18, y=1.02)

# Plot 1: Odia -> German
sns.kdeplot(data=df_dist_od, x='Baseline', fill=True, label='Baseline', clip=(0, 100), lw=3, ax=axes[0], color='C0')
sns.kdeplot(data=df_dist_od, x='Fine-Tuned', fill=True, label='Fine-Tuned', clip=(0, 100), lw=3, ax=axes[0], color='C1')
axes[0].set_title('Odia → German', fontsize=14)
axes[0].set_xlabel('Sentence BLEU Score', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].legend()

# Plot 2: German -> Odia
sns.kdeplot(data=df_dist_do, x='Baseline', fill=True, label='Baseline', clip=(0, 100), lw=3, ax=axes[1], color='C0')
sns.kdeplot(data=df_dist_do, x='Fine-Tuned', fill=True, label='Fine-Tuned', clip=(0, 100), lw=3, ax=axes[1], color='C1')
axes[1].set_title('German → Odia', fontsize=14)
axes[1].set_xlabel('Sentence BLEU Score', fontsize=12)
axes[1].set_ylabel('')
axes[1].legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Save the final model
# print("\nSaving the final bidirectional model...")
# trainer.save_model(FINAL_MODEL_PATH)
# print(f"Model saved to '{FINAL_MODEL_PATH}'")

## Visualization: Final Performance

To compare the final scores of the baseline vs. fine-tuned model.

In [ ]:
# --- 1. CONSOLIDATE RESULTS DATA ---
plot_data = [
    # --- Odia -> German Data ---
    {'Metric': 'BLEU', 'Direction': 'Odia → German', 'Model': 'Baseline', 'Score': baseline_ori_deu_metrics['bleu']},
    {'Metric': 'chrF', 'Direction': 'Odia → German', 'Model': 'Baseline', 'Score': baseline_ori_deu_metrics['chrf']},
    {'Metric': 'TER', 'Direction': 'Odia → German', 'Model': 'Baseline', 'Score': baseline_ori_deu_metrics['ter']},

    {'Metric': 'BLEU', 'Direction': 'Odia → German', 'Model': 'Fine-Tuned', 'Score': finetuned_ori_deu_metrics['bleu']},
    {'Metric': 'chrF', 'Direction': 'Odia → German', 'Model': 'Fine-Tuned', 'Score': finetuned_ori_deu_metrics['chrf']},
    {'Metric': 'TER', 'Direction': 'Odia → German', 'Model': 'Fine-Tuned', 'Score': finetuned_ori_deu_metrics['ter']},

    # --- German -> Odia Data ---
    {'Metric': 'BLEU', 'Direction': 'German → Odia', 'Model': 'Baseline', 'Score': baseline_deu_ori_metrics['bleu']},
    {'Metric': 'chrF', 'Direction': 'German → Odia', 'Model': 'Baseline', 'Score': baseline_deu_ori_metrics['chrf']},
    {'Metric': 'TER', 'Direction': 'German → Odia', 'Model': 'Baseline', 'Score': baseline_deu_ori_metrics['ter']},

    {'Metric': 'BLEU', 'Direction': 'German → Odia', 'Model': 'Fine-Tuned', 'Score': finetuned_deu_ori_metrics['bleu']},
    {'Metric': 'chrF', 'Direction': 'German → Odia', 'Model': 'Fine-Tuned', 'Score': finetuned_deu_ori_metrics['chrf']},
    {'Metric': 'TER', 'Direction': 'German → Odia', 'Model': 'Fine-Tuned', 'Score': finetuned_deu_ori_metrics['ter']},
]

df_plot = pd.DataFrame(plot_data)

In [ ]:
# --- 2. CREATE THE VISUALIZATION ---

# Set the theme
sns.set_theme(style="whitegrid")

# Create a figure and a set of subplots. This gives us explicit control.
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Define the two directions for the loop
directions = ['Odia → German', 'German → Odia']
palettes = ['viridis', 'plasma']

# Loop through the two axes and directions to create the plots
for i, direction in enumerate(directions):
  # Filter data for the current subplot
  subset_data = df_plot[df_plot['Direction'] == direction]

  # Create the bar plot on the specific axis (axes[i])
  sns.barplot(
      data=subset_data,
      x='Metric',
      y='Score',
      hue='Model', # 'Model' is responsible for the color distinction (Baseline vs Fine-tuned)
      palette=palettes[i],
      ax=axes[i]
  )

  # Set titles and labels for each subplot
  axes[i].set_title(f'Performance for {direction}', fontsize=15)
  axes[i].set_xlabel('')
  axes[i].set_ylabel('Score', fontsize=12)
  axes[i].tick_params(axis='x', labelsize=12)

  # Add the value annotations on top of the bars for each subplot
  for p in axes[i].patches:
    axes[i].annotate(f'{p.get_height():.2f}',
     (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha = 'center', va = 'center',
                     xytext = (0, 9),
                     textcoords = 'offset points',
                     fontsize=11, fontweight='bold')

  # Add legend for the current subplot
  axes[i].legend(title='Model Version', loc='upper right')


# Set a main title for the entire figure
fig.suptitle('Translation Performance: Baseline vs. Fine-Tuned Model', fontsize=20, y=1.03)

# Add the footnote text below the plots
fig.text(0.5, 0.01, '*For TER (Translation Edit Rate), a lower score is better.', ha='center', va='bottom', fontsize=12)

# Adjust layout to prevent labels/titles from overlapping
plt.tight_layout(rect=[0, 0.05, 1, 0.98]) # Adjust rect to make space for suptitle and footnote
plt.show()

## LOG FINAL RESULTS TO wandb

This code will take `df_results` DataFrame and log it as a beautiful, interactive table in wandb dashboard.

In [ ]:
import wandb

print("Initializing a new wandb run to log final results...")

wandb.init(
    project="Odia-German-Thesis",  # Groups all runs into a single project
    name="Final Performance Analysis", # Give this specific run a descriptive name
    job_type="evaluation"          # Categorize the run type
)


# LOG THE DATA
print("\nLogging final comparison results to Weights & Biases...")

# Create a wandb.Table object from results DataFrame
# We reset the index to make 'Metric' a column in the table
wandb_table = wandb.Table(dataframe=df_results.reset_index())

# Log the interactive table to new run
wandb.log({"final_performance_comparison": wandb_table})

# It's also good practice to update the run's summary with key final metrics
# This makes them easy to see and compare on the project dashboard
wandb.summary["final_bleu_ori_deu_baseline"] = baseline_ori_deu_metrics['bleu']
wandb.summary["final_bleu_ori_deu_finetuned"] = finetuned_ori_deu_metrics['bleu']
wandb.summary["final_bleu_deu_ori_baseline"] = baseline_deu_ori_metrics['bleu']
wandb.summary["final_bleu_deu_ori_finetuned"] = finetuned_deu_ori_metrics['bleu']

# FINISH THE RUN
wandb.finish()

print("\n✅ Final results successfully logged to your W&B dashboard.")

## Load the model

In [ ]:
# The path to the folder containing all the saved files
MODEL_PATH = "/content/drive/MyDrive/Thesis/model/nllb-odia-german-translator_model_final"

# LOAD THE FINE-TUNED MODEL AND TOKENIZER
print("Loading the fine-tuned tokenizer and model...")
# Load the fine-tuned tokenizer and model from the directory
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH, device_map="auto")

print("✅ Model and Tokenizer loaded successfully.")

# CREATE THE TRANSLATION PIPELINE
# Provide default language codes during initialization to satisfy the pipeline's requirement.
translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang=ODIA_LANG_CODE,
    tgt_lang=GERMAN_LANG_CODE,
)

print("✅ Translation pipeline created.")

In [ ]:
# PERFORM TRANSLATION
# --- Example 1: Odia to German (The Default Direction) ---
odia_input_text = "ଆଜି ପାଗ ବହୁତ ଭଲ ଅଛି।"
full_input_od = PREFIX_ORI_TO_DEU + odia_input_text

print(f"\nTranslating (Odia → German): '{odia_input_text}'")

german_translation = translator(full_input_od, max_length=128)
print(f"Output: {german_translation[0]['translation_text']}")

In [ ]:
# --- Example 2: German to Odia (Overriding the Default Direction) ---
german_input_text = "Wie ist das Wetter heute?"
full_input_de = PREFIX_DEU_TO_ORI + german_input_text

print(f"\nTranslating (German → Odia): '{german_input_text}'")

In [ ]:
# To translate in the reverse direction, override the languages directly in the call.
odia_translation = translator(
    full_input_de,
    src_lang=GERMAN_LANG_CODE,
    tgt_lang=ODIA_LANG_CODE,
    max_length=128
)
print(f"Output: {odia_translation[0]['translation_text']}")